In [ ]:
import numpy as np
from flask import Flask, request, jsonify, render_template
#from category_encoders.cat_boost import CatBoostEncoder
from sklearn import preprocessing
import pandas as pd
import pickle
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences

app = Flask(__name__)
main_model=tensorflow.keras.models.load_model("dialection_DL/MAIN_MODEL.h5")
gulf_model=tensorflow.keras.models.load_model("dialection_DL/gulf.h5")
maghreb_model=tensorflow.keras.models.load_model("dialection_DL/maghreb.h5")
nile_basin_model=tensorflow.keras.models.load_model("dialection_DL/Nile_Basin (1).h5")
levant_model=tensorflow.keras.models.load_model("dialection_DL/levant.h5")
tokenizer_main = pickle.load(open('dialection_DL/tokenizer_main.pkl','rb'))
tokenizer_gulf = pickle.load(open('dialection_DL/tokenizer_gulf.pkl','rb'))
tokenizer_maghreb = pickle.load(open('dialection_DL/tokenizer_maghreeb.pkl','rb'))
tokenizer_levant = pickle.load(open('dialection_DL/tokenizer_lavent.pkl','rb'))
tokenizer_nile_basin = pickle.load(open('dialection_DL/tokenizer_nile_basin.pkl','rb'))


@app.route('/')
def home():
    return render_template('index.html')

@app.route('/classify',methods=['POST'])
def classify():
    '''
    For rendering results on HTML GUI
    '''
    def splitolist(data):
        l=data.split(" ")
        return l
    int_features =request.form["sentence"]
    print(int_features)
    l=splitolist(int_features)
    seq=tokenizer_main.texts_to_sequences(l)
    b=[item for sublist in seq for item in sublist]
    #print(b)
    pad_seq=pad_sequences([b], maxlen=86,padding='post')
    prediction=main_model.predict(pad_seq).argmax(axis=1)
    
    target_names=['GULF','IQ','LEVANT', 'MAGHERB', 'NILE BASIN', 'YE']
    prediction=target_names[prediction[0]]
    print(prediction)
    if prediction=='MAGHERB':
        l=splitolist(int_features)
        seq=tokenizer_maghreb.texts_to_sequences(l)
        b=[item for sublist in seq for item in sublist]
        #print(b)
        pad_seq=pad_sequences([b], maxlen=61,padding='post')
        prediction = maghreb_model.predict(pad_seq).argmax(axis=1)
        target_names=['DZ','LY', 'MA', 'TN']
        prediction=target_names[prediction[0]]
    elif prediction=='LEVANT':
        l=splitolist(int_features)
        seq=tokenizer_levant.texts_to_sequences(l)
        b=[item for sublist in seq for item in sublist]
        #print(b)
        pad_seq=pad_sequences([b], maxlen=68,padding='post')
        prediction =levant_model.predict(pad_seq).argmax(axis=1)
        target_names=['JO', 'LB','PL', 'SY']
        prediction=target_names[prediction[0]]
    elif prediction=='NILE BASIN':
        l=splitolist(int_features)
        seq=tokenizer_nile_basin.texts_to_sequences(l)
        b=[item for sublist in seq for item in sublist]
        #print(b)
        pad_seq=pad_sequences([b], maxlen=54,padding='post')
        prediction =nile_basin_model.predict(pad_seq).argmax(axis=1)
        target_names=['EG', 'SD']
        prediction=target_names[prediction[0]]
    elif prediction=='GULF':
        l=splitolist(int_features)
        seq=tokenizer_gulf.texts_to_sequences(l)
        b=[item for sublist in seq for item in sublist]
        #print(b)
        pad_seq=pad_sequences([b], maxlen=86,padding='post')
        prediction =gulf_model.predict(pad_seq).argmax(axis=1)
        target_names=['AE','BH','KW','OM','QA', 'SA']
        prediction=target_names[prediction[0]]
    print(prediction)



    return render_template('index.html', prediction_text='dialection of this sentence is {}'.format(prediction))


if __name__ == "__main__":
    app.debug=False
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2022 01:04:22] "GET / HTTP/1.1" 200 -


"عامل ايه"
NILE BASIN


127.0.0.1 - - [14/Mar/2022 01:04:29] "POST /classify HTTP/1.1" 200 -


EG
"السلام يا حلوين مين يشرب"
LEVANT


...

"شوفت فيلم النهاردة"
NILE BASIN
EG


127.0.0.1 - - [14/Mar/2022 01:10:13] "POST /classify HTTP/1.1" 200 -


"انا كويسة"
LEVANT
PL


127.0.0.1 - - [14/Mar/2022 01:10:34] "POST /classify HTTP/1.1" 200 -


"اخباركوا ايه"
NILE BASIN
EG
